In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
# Imporvemnts
*# car health, car error (yes or no), car warning(10 different warnings)
# lambda function with relational database
# collecting data from different machine - collect from S3 - convert it to varios
*# - create mysql database + connect to database to AWS - 
# get data from twitter
*# get data from s3, cassandra, dynamo DB, postgres, mysql - access data from google analytics
# keep an eye on GO language
# draft letter to tableau for extension

# AWS Kinesis - Python + Spark

* Pre-requisites
* Introduction
* Installation
* CLI - useage

## Pre-requisites

* AWS Account
* Your own IAM role
* Python3 installed on system

## Introduction

AWS Kinesis is a **managed streaming service** by Amazon. It has 3 modules:
* Create streaming server using **Kinesis Data Stream** and ingest from various producers
* After ingesting data from producers persist to storage like S3, Redshit Spectrum, Dynamo DB or other datbase in AWS using **firehose**
* Perform real-time analytics on the stream using **Kinesis SQL Analytics**. 

Additioanlly these **Kinesis Data Modules** can be connected to other AWS servies such as EMR etc for other downstream applications.


![Kinesis Overview](https://docs.aws.amazon.com/streams/latest/dev/images/architecture.png)

## Installation

#### Step 1:  Install boto3 library to interact with AWS from python

In [ ]:
sudo pip3 install --upgrade boto3 

#### Step 2:  Install AWS CLI library to interact with AWS form console

In [ ]:
sudo pip3 install --upgrade awscli

## What exactly is Kinesis Data Stream?

* Think of Kinesis Data Stream as a computer(s) into which you can write data and someone else can read that data.
* For example,

    Consider Dublin Bus :
        
    The buses **produce** their location information using GPS and send it to **computer(s)** located in AWS's datacenter. 
    
    Then we can **consume** it from there for analytics purpose and extract the ETA for each bus stop.

## Exercsise 1 - Create Kinesis Stream

We will create a stream called **taxi_fleet_stream**. We can use AWS CLI or Python for this purpose, both ways are show below!

### Create stream with AWS CLI

We will create an AWS Kinesis Data Stream to store the data which we will recieive from the bus(s). Takes about 30 sec to create the stream!

In [2]:
## create stream
!aws kinesis create-stream  --stream-name 'taxi_fleet_stream' --shard-count 1

In [3]:
## list stream
!aws kinesis list-streams

{
    "StreamNames": [
        "taxi_fleet_stream"
    ]
}


In [5]:
# describe stream
!aws kinesis describe-stream --stream-name 'taxi_fleet_stream'

{
    "StreamDescription": {
        "Shards": [
            {
                "ShardId": "shardId-000000000000",
                "HashKeyRange": {
                    "StartingHashKey": "0",
                    "EndingHashKey": "340282366920938463463374607431768211455"
                },
                "SequenceNumberRange": {
                    "StartingSequenceNumber": "49593832700362594935654438540330927680073445288559247362"
                }
            }
        ],
        "StreamARN": "arn:aws:kinesis:eu-west-1:637994078207:stream/taxi_fleet_stream",
        "StreamName": "taxi_fleet_stream",
        "StreamStatus": "ACTIVE",
        "RetentionPeriodHours": 24,
        "EnhancedMonitoring": [
            {
                "ShardLevelMetrics": []
            }
        ],
        "EncryptionType": "NONE",
        "KeyId": null,
        "StreamCreationTimestamp": 1552569737.0
    }
}


In [17]:
## destroy stream
!aws kinesis delete-stream --stream-name "taxi_fleet_stream"

### Create stream with python 

We will create an AWS Kinesis Data Stream using python boto3

In [ ]:
import boto3
kinesis_client = boto3.client('kinesis', region_name='eu-west-1')

# create stream
kinesis_client.create_stream(StreamName='taxi_fleet_stream',ShardCount=1)

## If stream already exists, then you will get a "ResourceInUseException"

In [ ]:
# list stream
kinesis_client.list_streams()

In [ ]:
# destory stream
kinesis_client.delete_stream(StreamName = 'taxi_fleet_stream')

### Exercise 2 - Create Producer

The taxi will generate its own carID, fuel level, timestamp as data. We will use python for creating a producer.

Before sending the data we generated, we must create it to a **record**
Each Record should have 2 attributes, data and partiton key
Data : The JSON Dump of the data taht you want to send
PartitonKey : String to indicate on which shard this record is sumbitted

![Producer work flow](./images/producer_workflow.png)

In [11]:
from pprint import pprint
import boto3
import json
import random
import time
from datetime import datetime


# initialize client
kinesis_client = boto3.client('kinesis', region_name='eu-west-1')

# generate the data we want to send
data = {
    "time" : str(datetime.now()),
    "car_id" : 1,
    "fuel" : random.randint(1, 100)
}

# convert it to record
record = {}
record['Data'] = json.dumps(data)
record['PartitionKey'] = str(data['car_id'])

pprint(record)

{'Data': '{"time": "2019-03-11 14:03:53.986702", "fuel": 52, "car_id": 1}',
 'PartitionKey': '1'}


After you send the record a sequence number is assigned automatically by Kinesis. It is unique for each record

In [12]:
# send data to kinesis - put record
response = kinesis_client.put_record(**record, StreamName='taxi_fleet_stream')
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '110',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Mon, 11 Mar 2019 14:03:57 GMT',
                                      'x-amz-id-2': 'OtBDTTEoF4kfsfPseAWGd8TZoFhpspNbzSZW5EgpAc1HOcwovE1zNPVOL/0xwIMv0b0Ju3GUKn2E1UxnLUVXQBplx/DNNVXU',
                                      'x-amzn-requestid': 'd73ddda0-6872-2310-823d-b0ad76616a98'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd73ddda0-6872-2310-823d-b0ad76616a98',
                      'RetryAttempts': 0},
 'SequenceNumber': '49593741073915218598441994360557169733085073430991077378',
 'ShardId': 'shardId-000000000000'}


We will generate this record every 1 second and send it

In [ ]:
record = {}
## send data every 1 second
while True:
    ## create the data we want to send
    data = {
        "time" : str(datetime.now()),
        "car_id" : 1,
        "fuel" : random.randint(1, 100)
    }
    
    # convert it to record
    record['Data'] = json.dumps(data)
    record['PartitionKey'] = str(data['car_id'])
    
    ## send the record
    response = kinesis_client.put_record(**record,StreamName='taxi_fleet_stream')
    pprint(record)
    print("\n\n")
    pprint(response)
    print("\n====================\n")
    record = {}
    
    ## sleep for 1 second - then send next record
    time.sleep(1)


In [ ]:
## check payload size
import sys
print ("Estimated size: " + str(sys.getsizeof(record) / 1024) + "KB")

But Sending small chunks of data every 1 second is very expensive, AWS Kinesis charges you for every 1 Million PUT requests (**Each PUT request is defined as record of size 25KB or less**). 

In out case, we have a record size of 0.28125 KB, which is counted as 1 PUT request, as it is less that 25KB. 

You can see that if we have 1000s of taxis sending data every second of such small size, then we will quickly eat up our 1 Million **PUT request** Limit. We will explore another option called **put_records()**, whcih allows us to aggregate records and send them as a batch!

Note : If a single record was of say 40KB, then it would count as 2 PUT request and so on!

### put_records() - Send more than 1 record

In [15]:
## send data when we have 10 records,
RECORD_LIMIT = 10
record_set = [] # holds all 10 records
record = {}
while True:
    ## create the data we want to send
    data = {
        "time" : str(datetime.now()),
        "car_id" : 1,
        "fuel" : random.randint(1, 100)
    }
    
    # convert it to record
    record['Data'] = json.dumps(data)
    record['PartitionKey'] = str(data['car_id'])
    record
    
    ## send the data as a batch, send them togather
    if(len(record_set) == RECORD_LIMIT):
        response = kinesis_client.put_records(StreamName='taxi_fleet_stream', 
                                              Records = record_set
                                              )    
        pprint(record_set)
        print("\n\n")
        pprint(response)
        print("\n====================\n")
        record_set = [] #clear payload_set
    
    
    record_set.append(record)
    record = {} # clear record
    
    ## sleep for 1 second - then generate next record
    time.sleep(1)


[{'Data': '{"time": "2019-03-11 14:27:21.776486", "fuel": 99, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:22.777764", "fuel": 81, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:23.779160", "fuel": 7, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:24.780509", "fuel": 97, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:25.781794", "fuel": 91, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:26.783138", "fuel": 68, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:27.783773", "fuel": 16, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:28.785116", "fuel": 93, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:29.786417", "fuel": 34, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:30.787758", "fuel": 27, "car_id": 1}',
  'PartitionKey

[{'Data': '{"time": "2019-03-11 14:27:52.136345", "fuel": 78, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:53.294188", "fuel": 38, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:54.295649", "fuel": 80, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:55.297099", "fuel": 56, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:56.298555", "fuel": 15, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:57.300021", "fuel": 96, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:58.301483", "fuel": 7, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:27:59.302917", "fuel": 13, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:00.304283", "fuel": 43, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:01.305724", "fuel": 67, "car_id": 1}',
  'PartitionKey

[{'Data': '{"time": "2019-03-11 14:28:22.901708", "fuel": 76, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:24.024308", "fuel": 92, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:25.025770", "fuel": 51, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:26.027137", "fuel": 61, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:27.028544", "fuel": 69, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:28.029747", "fuel": 25, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:29.031167", "fuel": 56, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:30.032500", "fuel": 94, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:31.033825", "fuel": 26, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:32.035146", "fuel": 77, "car_id": 1}',
  'PartitionKe

[{'Data': '{"time": "2019-03-11 14:28:53.440510", "fuel": 36, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:54.621774", "fuel": 89, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:55.623235", "fuel": 29, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:56.624570", "fuel": 18, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:57.626004", "fuel": 58, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:58.627377", "fuel": 29, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:28:59.628746", "fuel": 66, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:00.630190", "fuel": 4, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:01.631637", "fuel": 78, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:02.633067", "fuel": 14, "car_id": 1}',
  'PartitionKey

[{'Data': '{"time": "2019-03-11 14:29:24.270864", "fuel": 23, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:25.435930", "fuel": 62, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:26.437325", "fuel": 80, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:27.438739", "fuel": 56, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:28.440171", "fuel": 17, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:29.441622", "fuel": 56, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:30.442564", "fuel": 21, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:31.443923", "fuel": 98, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:32.445369", "fuel": 14, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:29:33.446776", "fuel": 17, "car_id": 1}',
  'PartitionKe

...

[{'Data': '{"time": "2019-03-11 14:43:43.312237", "fuel": 3, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:43:44.483184", "fuel": 14, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:43:45.484634", "fuel": 10, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:43:46.485820", "fuel": 5, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:43:47.487227", "fuel": 18, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:43:48.488556", "fuel": 48, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:43:49.489956", "fuel": 80, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:43:50.491258", "fuel": 46, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:43:51.492587", "fuel": 33, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:43:52.493942", "fuel": 94, "car_id": 1}',
  'PartitionKey'

[{'Data': '{"time": "2019-03-11 14:44:14.198521", "fuel": 90, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:15.350529", "fuel": 13, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:16.351791", "fuel": 48, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:17.353146", "fuel": 51, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:18.354526", "fuel": 75, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:19.355823", "fuel": 95, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:20.357265", "fuel": 25, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:21.358646", "fuel": 44, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:22.360013", "fuel": 18, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:23.361349", "fuel": 47, "car_id": 1}',
  'PartitionKe

[{'Data': '{"time": "2019-03-11 14:44:44.670303", "fuel": 69, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:45.839335", "fuel": 25, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:46.840676", "fuel": 43, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:47.842025", "fuel": 32, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:48.843402", "fuel": 34, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:49.844769", "fuel": 47, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:50.846244", "fuel": 93, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:51.847760", "fuel": 43, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:52.849069", "fuel": 50, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:44:53.850381", "fuel": 6, "car_id": 1}',
  'PartitionKey

[{'Data': '{"time": "2019-03-11 14:45:15.206560", "fuel": 83, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:45:16.370788", "fuel": 97, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:45:17.372117", "fuel": 12, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:45:18.373419", "fuel": 73, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:45:19.374873", "fuel": 29, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:45:20.376323", "fuel": 58, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:45:21.377770", "fuel": 77, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:45:22.379222", "fuel": 60, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:45:23.380557", "fuel": 24, "car_id": 1}',
  'PartitionKey': '1'},
 {'Data': '{"time": "2019-03-11 14:45:24.381903", "fuel": 77, "car_id": 1}',
  'PartitionKe

KeyboardInterrupt: 

* Each put_records() call can support up to 500 records in the record_set(). 
* Each record should have its own partionon key specified as its own part.
* Each record in the request can be as large as 1 MB, up to a limit of 5 MB for the entire request, including partition keys.

Explicitely mention the Data and Partiton key for each record

### Exercise 3 : Create Consumer

We will create a consumer in python who will read this data from kinesis

In [ ]:
import boto3
from pprint import pprint
import json
from datetime import datetime
import time


# intitalize client
kinesis_client = boto3.client('kinesis', region_name='eu-west-1')

response = kinesis_client.describe_stream(StreamName='taxi_fleet_stream')
pprint(response)
# # pprint(response)
# my_shard_id = response['StreamDescription']['Shards'][0]['ShardId']

# shard_iterator = kinesis_client.get_shard_iterator(StreamName=my_stream_name,
#                                                       ShardId=my_shard_id,
#                                                       ShardIteratorType='LATEST')

# my_shard_iterator = shard_iterator['ShardIterator']

# record_response = kinesis_client.get_records(ShardIterator=my_shard_iterator)

# while 'NextShardIterator' in record_response:
#     record_response = kinesis_client.get_records(ShardIterator=record_response['NextShardIterator'])
#     if(len(record_response['Records'])!= 0):
#         [print(record_response['Records'])

    
#     print("\n\n\n")
#     time.sleep(1)


In [ ]:
kinesis_client.describe_stream(st my_stream_name)